In [2]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# elem.send_keys(Keys.ENTER)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import re
import time
from datetime import datetime
import pickle

import pymysql 
import pandas as pd 
import numpy as np
from tqdm import tqdm

# # save data
# with open('urls.pickle','wb') as fw:
#     pickle.dump(myset.urls, fw)

# # load data
# with open('urls.pickle', 'rb') as fr:
#     urls = pickle.load(fr)

In [92]:
class createDB():
    def __init__(self, years=None, genres={'건강취미':55890, '경제경영':170, '고전':2105, '과학':987, 
                                       '사회과학':798, '소설시희곡':1, '에세이':55889, '역사':74, 
                                       '인문학':656, '자기계발':336, '장르소설':112011, '예술대중문화':517}): # '청소년':1137
        if years is None: self.years = [i for i in range(1999, 2023)]
        else: self.years = years
        self.genres = genres
        self.urls, self.dfs = {}, {}
        
    def open_driver(self, url, scroll=False):
        user_agent = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        options = webdriver.ChromeOptions()
        options.add_argument(user_agent)
        options.add_argument('headless') # 페이지 안 열기 
        options.add_argument('window-size=1920x1080') # 사이즈 지정
        options.add_argument("disable-gpu") # 안 보이게
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(1)
        self.driver.get(url)
        time.sleep(0.5)
        
        if scroll:
            prev_height = self.driver.execute_script('return document.body.scrollHeight')
            while True:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(1)
                curr_height = self.driver.execute_script('return document.body.scrollHeight')
                if prev_height == curr_height: break
                prev_height = curr_height
        
    def save_urls(self, genre):
        self.urls[genre] = {}
        tqdm_years = tqdm(self.years)
        for year in tqdm_years:
            for i in range(1,7): # 한 페이지 당 50권, 6페이지 반복
                url = 'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=YearlyBest&BranchType=1&CID={}&Year={}&page={}&cnt=300&SortOrder=1'.format(self.genres[genre], year, i)
                self.open_driver(url)
                elems = self.driver.find_elements_by_class_name('bo3')
                for elem in elems:
                    title = elem.find_element_by_tag_name('b').text
                    self.urls[genre][title] = elem.get_attribute('href')
            tqdm_years.set_description(genre+' '+str(year)+' urls')
        tqdm_years.close()

    def connect_mysql(self, connect=True):
        if connect:
            self.db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
            
    def create_tbl(self, genre):
        self.connect_mysql()
        sql = """CREATE TABLE IF NOT EXISTS yes24_{}(
                title VARCHAR(100), author VARCHAR(100),
                genre_1 VARCHAR(200), genre_2 VARCHAR(300), 
                rank SMALLINT, rate DOUBLE, count SMALLINT,
                describe VARCHAR(1000)
                );""".format(genre)
        self.cursor.execute(sql)
        self.connect_mysql(False)
    
    def fill_tbl(self, genre):
        self.connect_mysql()
        tqdm_book = tqdm(self.urls[genre].items())
        for book_title, book_info in tqdm_book:
            try: self.open_driver(book_info['url'], scroll=True)
            except: continue
                
            title = book_title
#             title = self.driver.find_element_by_class_name('prod_title').text
            author = book_info['author']
            rank = book_info['rank']
            
            elems = self.driver.find_elements_by_class_name('category_list_item')
            genre_1, genre_2 = '', ''
            for elem in elems:
                gs = elem.find_elements_by_class_name('intro_category_link')
                for i, g in enumerate(gs):
                    if i==1: genre_1 += g.text+', '
                    elif i==2 or i==3: genre_2 += g.text+', '
            
            describe = self.driver.find_elements_by_class_name('info_text')[1].text

            sql = '''INSERT INTO gyoby_{0} VALUES(
            "{1}", "{2}", "{3}", "{4}", {5}, {6}, {7}, "{8}");
            '''.format(genre, title, author, genre_1, genre_2, rank, rate, count, describe)
            self.cursor.execute(sql)
            self.db.commit()
            tqdm_book.set_description(genre+' insert data')
        self.connect_mysql(False)
        tqdm_book.close()
        
    def get_df(self, genre):
        self.connect_mysql(True)
        sql = 'SELECT * FROM gyobo_{}'.format(genre)
        self.cursor.execute(sql)
        self.dfs[genre] = pd.DataFrame(self.cursor.fetchall())
        self.dfs[genre].to_csv('gyobo_{}.csv'.format(genre), index=False)
        self.connect_mysql(False)
        print('save gyobo_'+genre+'.csv')
    
    def auto(self, genre):
        self.save_urls(genre)
        self.create_tbl(genre)
        self.fill_tbl(genre)
        self.get_df(genre)

In [93]:
years = [i for i in range(2015, 2023)]
myset = createDB(years)
print(myset.years)
print(myset.genres)
# for genre in myset.urls.keys():
#     myset.auto(genre)

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
{'건강취미': 55890, '경제경영': 170, '고전': 2105, '과학': 987, '사회과학': 798, '소설시희곡': 1, '에세이': 55889, '역사': 74, '인문학': 656, '자기계발': 336, '장르소설': 112011, '예술대중문화': 517}


In [47]:
for genre in myset.genres:
    myset.save_urls(genre)

청소년 2022 urls: 100%|██████████████████████████████████████████████████████████████████| 8/8 [06:18<00:00, 47.32s/it]/s]
